# How to learn feature for functional maps

In this notebook, we show how to use deep functional maps to learn feature for 3d shape matching.

In [19]:
# Set the backend for geomstats to PyTorch, commented for github test
import os

os.environ["GEOMSTATS_BACKEND"] = "pytorch"

import torch
from torch.utils.data import random_split

from geomfum.convert import P2pFromFmConverter
from geomfum.dataset.torch import PairsDataset, ShapeDataset
from geomfum.descriptor.learned import FeatureExtractor
from geomfum.descriptor.spectral import WaveKernelSignature
from geomfum.forward_functional_map import ForwardFunctionalMap
from geomfum.learning.losses import (
    BijectivityLoss,
    GeodesicError,
    LaplacianCommutativityLoss,
    LossManager,
    OrthonormalityLoss,
)
from geomfum.learning.models import FMNet
from geomfum.learning.trainer import DeepFunctionalMapTrainer

First, we define our model. We can instantiate it combining feature extractors and forward logic, however, we provide some classic frameworks, like Functional Map network.

In [20]:
# Build the model
fmap_module = ForwardFunctionalMap(1e3, 1, True)

feature_extractor = FeatureExtractor.from_registry(
    which="diffusionnet",
    device="cuda",
    k_eig=200,
    in_channels=128,
    descriptor=WaveKernelSignature(n_domain=128),
)

functional_map_model = FMNet(
    feature_extractor=feature_extractor,
    fmap_module=fmap_module,
    converter=P2pFromFmConverter(),
)


Then, we instantiate the training dataset. \
In our Datset class, we cna set boolean variable to specify what kind of objects we expect in the dataset.\
In the dataset folder, we always expect datas to be stored in a 'shapes' folder. \
If we have access to tamplate ground thruth correspondences, we can set correspondences= True, in this case we expect to have a folder called 'corr'.
We can set spectral=True if we want to compute spectral quantities, and set distances=True if we want to compute distances, this is expensive, so we suggest to do so only for testing dataset.

In [ ]:
TRAIN_SET_PATH = "../../../datasets/faust/test_set/"
dataset = ShapeDataset(
    TRAIN_SET_PATH, spectral=True, distances=False, device="cuda", k=30
)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_shapes, validation_shapes = random_split(dataset, [train_size, val_size])


train_dataset = PairsDataset(
    train_shapes,
    pair_mode="all",
)

validation_dataset = PairsDataset(
    validation_shapes,
    pair_mode="all",
)


C:\Users\giuli\OneDrive\Research\geomfum_proj\geomfum\geomfum\_backend\pytorch\sparse.py:22: UserWarning: Sparse CSC tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\SparseCsrTensorImpl.cpp:55.)
  return _torch.sparse_csc_tensor(ccol_indices, row_indices, values, size=array.shape)


Sometimes the distance computation is usefull only at validation time, so we suggest to perform the following trick

In [ ]:
from torch.utils.data import Subset


TRAIN_SET_PATH = "../../../datasets/faust/test_set/"
dataset1 = ShapeDataset(
    TRAIN_SET_PATH,
    spectral=True,
    distances=False,
    correspondences=False,
    device="cuda",
    k=30,
)
dataset2 = ShapeDataset(
    TRAIN_SET_PATH,
    spectral=True,
    distances=True,
    correspondences=True,
    device="cuda",
    k=30,
)

train_size = int(0.8 * len(dataset1))
val_size = len(dataset1) - train_size

train_shapes, validation_shapes = random_split(dataset1, [train_size, val_size])
# Create the full list of indices and shuffle
train_indices = train_shapes.indices
val_indices = validation_shapes.indices

train_shapes = Subset(dataset1, train_indices)
validation_shapes = Subset(dataset2, val_indices)

train_dataset = PairsDataset(
    train_shapes,
    pair_mode="all",
)

validation_dataset = PairsDataset(
    validation_shapes,
    pair_mode="all",
)


Then , we instantiate the optimizer

In [22]:
optimizer = torch.optim.Adam(functional_map_model.parameters(), lr=1e-3)

Now we define the losses that we will consider. Again we can define our own losses, however we provide some classic functional map energies, like the orthonormality loss. 
\
For evaluation, we can use training losses, or we can compute the geodesic distance loss, to evaluate the estimates.\
We note that this loss makes sense only if we ahve access to a ground thruth correspondence or if the shapes share the same triangulation.

In [23]:
# define the loss
losses = [
    OrthonormalityLoss(weight=1.0),
    BijectivityLoss(weight=1.0),
    LaplacianCommutativityLoss(weight=1e-3),
]
loss_manager = LossManager(losses)

losses = [
    GeodesicError(),
]

val_loss_manager = LossManager(losses)

We have defined a trainer for simplicity that thakes as input model, losses, train and val datasets and optimizer and manages the training loops.

In [24]:
trainer = DeepFunctionalMapTrainer(
    model=functional_map_model,
    train_loss_manager=loss_manager,
    val_loss_manager=val_loss_manager,
    train_set=train_dataset,
    val_set=validation_dataset,
    optimizer=optimizer,
    device="cuda",
    epochs=10,
)

In [26]:
trainer.train()

INFO: Epoch [1/10] - Training
Epoch 1/10 (Train):   0%|          | 0/240 [00:00<?, ?batch/s]


KeyError: 'tr_reg_094.off'

In [ ]:
data1 = train_dataset[0]
mesh1 = data1["source"]["mesh"]
mesh2 = data1["target"]["mesh"]

In [ ]:
a = trainer.model.feature_extractor(mesh1)

In [ ]:
import geomstats.backend as gs

In [ ]:
gs.array(a.squeeze().double().T)

tensor([[-0.3249, -0.2951, -0.4287,  ..., -0.3962, -0.3062, -0.4485],
        [-0.3282, -0.4835, -0.3857,  ..., -0.3310, -0.4567, -0.3650],
        [-0.0126, -0.0301,  0.0446,  ...,  0.0626, -0.0043,  0.0265],
        ...,
        [-0.1333, -0.1450, -0.1222,  ..., -0.1283, -0.0805, -0.1357],
        [-0.1456, -0.1540, -0.2034,  ..., -0.3405, -0.2318, -0.2368],
        [ 0.0217, -0.0133,  0.0037,  ..., -0.0888, -0.0129, -0.0303]],
       device='cuda:0', grad_fn=<CloneBackward0>)